In [25]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup ,NavigableString, Comment
import html5lib

In [341]:
def get_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,  'html.parser')
    article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find_all('p', recursive = False)
    h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
    h2_text = '\n'.join(line.strip() for line in h2.get_text().split('\n') if line.strip())
    # Update the text of the h2 tag
    h2.string = h2_text
    for child in article[:-4]:
        imgs = child.find_all('img',recursive=True)
        print(len(imgs))
        a_tags = child.find_all('a')
        if len(a_tags) > 0:
            for a_tag in a_tags:
                try:
                    a_text = a_tag.text
                    parent_a_tag =  a_tag.parent
                    parent_a_tag.string = parent_a_tag.text
                except AttributeError:
                    break
            
        if len(imgs) >0:
            for t in imgs:
                try:
                    #t is image element
                    p_next = t.parent.next_sibling.next_sibling
                    #p_next.decompose()
                    #p_next['style'] = "text-align: center;"
                    caption_tag = soup.new_tag('caption')
                    t.insert_before(caption_tag)
                    caption_tag.append(t)
                    caption_tag['width'] = 800
                    caption_tag['align'] ="aligncenter"
                    caption_tag.append(p_next.text)
                    try:
                        t.attrs.pop('class')
                        t.attrs.pop('onclick')
                        t.attrs.pop('width')
                        t['src'] = t.get('data-original', None)
                        t['width'] = 800
                        t['height'] = 400
                    except KeyError:
                        break
                except AttributeError:
                    print(t)
    soup2 = BeautifulSoup("", 'html.parser')
    soup2.append(h2)
    for i in article[:-4]:
        if 'class' not in i.attrs:
            soup2.append(i)
        if i.string == "":
            i.decompse
        
    content = soup2.prettify()
    return content



In [148]:
#soup2 = BeautifulSoup("", 'html.parser')

def get_content2(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,  features='html')

    #remove the last 4 element
    article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
    
    children = article.find_all(recursive=False)
    for i in children[-4:]:
        i.decompose()
    caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
    #find a,span and remove tag with text
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    # set attribut for img
    b = article.find_all('img')
    for i in b:
        if '.svg' in i['src']:
            i.decompose()
    for i in article.find_all('img'):
        if 'data-original' in i.attrs.keys():
            i['src'] = i.get('data-original')
        #i.attrs = ['class', 'alt', 'src', 'data-original']
        del i['onclick']
        del i['style']
        del i['class']
    img_list = article.find_all('img')
    n_img = len(img_list)
    #print(len(caption_text_list))
    #create caption for the images
    for i in range(0,n_img):
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        caption_text = NavigableString(caption_text_list[i].string)
        caption_end = NavigableString("[/caption]")
        caption_text_list[i].decompose()
        # Insert the custom tags and caption text around the <img> tag
        img_list[i].insert_before(caption_start)
        img_list[i].insert_after(caption_end)
        try:
            img_list[i].insert_after(caption_text) 
        except IndexError as e:
            print(e)
        
        """caption_tag = soup.new_tag('caption')
        img_list[i].insert_before(caption_tag)
        caption_tag.append(img_list[i])
        caption_tag['width'] = 800
        caption_tag['align'] ="aligncenter"
        try:
            caption_tag.append(caption_text_list[i].string)
        except IndexError as e:
            print(e)
            #create caption"""
            
        #print(i.attrs.keys())"""
    #remove js  and css
    h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
    h2_text = '\n'.join(line.strip() for line in h2.get_text().split('\n') if line.strip())
    # Update the text of the h2 tag
    h2.string = h2_text
    #soup2 = BeautifulSoup("", 'html.parser')
    #soup2.append(h2)
    for script_or_style in article(['script', 'style']):
        script_or_style.decompose()
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            del i['id']
            del i['class']
            del i['style']
        except AttributeError:
            continue
        try:
            print(i)
        except TypeError:
            continue
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
    for video in article.find_all('video'):
        video.decompose()
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag
# Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    for i in article.find_all('div', {'align': 'center'}):
        i.decompose()
    for element in article.find_all(recursive = True,string=True):
        if isinstance(element, NavigableString) and element.strip() == '':
            element.extract()
    for i in article.find_all('div'):
        if i.children == None and i.string == None:
            i.decompose()
    return article

In [149]:
test = get_content2('https://www.24h.com.vn/ban-tre-cuoc-song/cau-be-lop-9-truong-lang-duoc-tuyen-thang-vao-dai-hoc-so-1-chau-a-co-tri-nho-sieu-pham-kha-nang-tu-duy-logic-tuyet-dinh-c64a1521211.html')

<h2>Thành tích học tập của cậu bé này đã khiến không chỉ bạn bè cùng trang lứa mà thậm chí những người tiền bối đã học cao hơn cậu cũng phải ngưỡng mộ.</h2>
<p>Cậu bé 14 tuổi Quan Tử Khâm hiện đang là học sinh lớp 9 tại một trường trung học ở huyện Bác Bạch, tỉnh Quảng Tây, phía Nam Trung Quốc. Cuối năm 2022, Quan Tử Khâm đã vinh dự lọt vào danh sách trúng tuyển của Chương trình đào tạo tài năng hàng đầu về khoa học toán học của Đại học Thanh Hoa danh giá. </p>
<p>Điều này có nghĩa là cậu bé lớp 9 sẽ "nhảy cấp". Đây được xem là thành tích chưa từng có tiền lệ trước đó. Đã từng có trường hợp những học sinh lớp 1,2 nhảy cấp lên lớp 4,6, nhưng việc nhảy cấp từ trung học lên thẳng Đại học và đặc biệt là Thanh Hoa là tỷ lệ rất hiếm thấy. </p>
<p align="center"></p>
<div>
<div style="position:relative;max-width: 740px;margin: 0 auto">
<div><section class="txtCent bnrPtn inImgExpandSection" id="vnapzJKUhp_213_15s_container"><div id="vnapzJKUhp_213_15s">
</div>
</section>[caption id="" align="

In [60]:
response = requests.get('https://www.24h.com.vn/cong-nghe-thong-tin/apple-gay-soc-sap-cho-nguoi-dung-android-giao-tiep-voi-imessage-c55a1519650.html')
soup = BeautifulSoup(response.content, 'html.parser')
#remove the last 4 element
article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
children = article.find_all(recursive=False)
for i in children[-4:]:
    i.decompose()
#find a,span and remove tag with text
tags_to_remove = article.find_all(['a', 'span'])
for tag in tags_to_remove:
    # Extract the text from the tag
    tag_text = tag.get_text()
    # Replace the tag with its text content
    tag.replace_with(tag_text)
    tag.text.strip()
# set attribut for img
b = article.find_all('img')
for i in b:
    if '.svg' in i['src']:
        i.decompose()
for i in article.find_all('img'):
    if 'data-original' in i.attrs.keys():
        i['src'] = i.get('data-original')
    #i.attrs = ['class', 'alt', 'src', 'data-original']
    del i['onclick']
    del i['style']
    del i['class']
    i['width'] = 800
    i['height'] = 400

img_list = article.find_all('img')
n_img = len(img_list)
#print(n_img)
#print(len(caption_text_list))
for i in range(0,n_img):
        #create caption
        caption_tag = soup.new_tag('caption')
        img_list[i].insert_before(caption_tag)
        caption_tag.append(img_list[i])
        caption_tag['width'] = 800
        caption_tag['align'] ="aligncenter"
        caption_tag.append(caption_text_list[i])
    #print(i.attrs.keys())
#remove js  and css
h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
soup2 = BeautifulSoup("", 'html.parser')
soup2.append(h2)
for script_or_style in article(['script', 'style']):
    script_or_style.decompose()
#remove id and class from attribute
for i in article.find_all(recursive = True):
    try:
        del i['id']
        del i['class']
    except AttributeError:
        continue
    try:
        print(i)
        soup2.append(i)
    except TypeError:
        continue
        
#for i in article.find_all('p',class_='img_chu_thich_0407'):
    #i.decompose()                


<p>Sau nhiều năm chịu sức ép bởi Google nhằm buộc Apple phải hỗ trợ giao thức tin nhắn RCS trong iMessage, nhà sản xuất iPhone cuối cùng cũng phải chấp nhận khi cam kết thực hiện điều này vào năm 2024.</p>
<p style="text-align: center;"><div id="container-24h-banner-in-image" style="text-align: center;width: 100%;clear: both;position: relative;">
<div style="position:relative;max-width: 740px;margin: 0 auto">
<div><section class="txtCent bnrPtn inImgExpandSection" id="JOyNWkLerd_213_15s_container"><div id="JOyNWkLerd_213_15s">
</div>
</section><caption align="aligncenter" width="800"><img alt="Apple gây sốc, sắp cho người dùng Android “giao tiếp” với iMessage - 1" height="400" src="https://image-us.24h.com.vn/upload/4-2023/images/2023-11-17/2-1700203824-539-width740height416.jpg" width="800"/><p class="img_chu_thich_0407" style="text-align: center;">RCS cuối cùng cũng sắp có khả năng giao tiếp với iMessage.</p></caption></div>
<div id="24h-banner-in-image" style="left: 50%;transform: t